In [17]:
# 导入必要的库
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import torch
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
import torch.nn.functional as F
import pandas as pd
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests

In [18]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [19]:
num_class=2
adj_parameter = 10
dim_he_list = [200,200,100]
gcn_dopout=0.5
classifier_out_dim=2
num_epoch=500
test_inverval=20

In [20]:
def filter_features(data, variance_threshold=0.001):
    """
    过滤掉没有信号（零平均值）或低方差的特征。

    参数:
    data: numpy array 或 pandas DataFrame
    variance_threshold: 方差阈值 (默认1e-5)

    返回:
    过滤后的数据
    """
    # 计算均值和方差
    mean_values = data.mean(axis=0)
    variances = data.var(axis=0)

    # 筛选条件：均值不为零且方差大于阈值
    mask = (mean_values != 0) & (variances > variance_threshold)

    # 过滤特征
    filtered_data = data[:, mask] if isinstance(data, np.ndarray) else data.loc[:, mask]

    return filtered_data

In [21]:
def select_features(X, y, alpha=0.05, variance_threshold=0.50):
    """
    使用ANOVA F值和PCA进行特征选择。

    参数:
    X: 特征矩阵 (numpy array 或 pandas DataFrame)
    y: 标签向量 (numpy array 或 pandas Series)
    alpha: 显著性水平 (默认0.05)
    variance_threshold: 第一个主成分需解释的最大方差比例 (默认0.50)

    返回:
    选择的特征 (numpy array 或 pandas DataFrame)
    """
    # 标准化特征
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 进行方差分析
    F_values, p_values = f_classif(X_scaled, y)

    # FDR 校正
    _, p_corrected, _, _ = multipletests(p_values, method='fdr_bh')

    # 选择显著性特征
    significant_features = np.where(p_corrected < alpha)[0]

    # 使用 PCA 选择特征
    pca = PCA()
    pca.fit(X_scaled)

    # 检查第一个主成分的方差解释
    explained_variance = pca.explained_variance_ratio_

    # 选择特征数量
    num_features = np.sum(explained_variance < variance_threshold)

    # 根据显著性选择特征
    selected_feature_indices = significant_features[:num_features]

    # 返回选择的特征
    return X.iloc[:, selected_feature_indices] if isinstance(X, pd.DataFrame) else X[:, selected_feature_indices]


In [22]:
def scale_to_unit_interval(data):
    """
    将数据缩放到 [0, 1] 的区间。

    参数:
    data: numpy array 或 pandas DataFrame

    返回:
    缩放后的数据
    """
    # 计算最小值和最大值
    min_val = data.min(axis=0)
    max_val = data.max(axis=0)
    
    # 进行线性变换
    scaled_data = (data - min_val) / (max_val - min_val)
    
    return scaled_data

In [23]:
Clinical = pd.read_csv('../../DATA/KIPAN/Human__TCGA_KIPAN__MS__Clinical__Clinical__01_28_2016__BI__Clinical__Firehose_common_samples.tsv', sep='\t')
Clinical

,Unnamed: 0,years_to_birth,Tumor_purity,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,histological_type,number_of_lymph_nodes,PAM50,...,PR.Status,HER2.Status,gender,radiation_therapy,race,ethnicity,Median_overall_survival,overall_survival,status,overallsurvival
0,TCGA.D8.A1JH,56.0,0.6126,stagei,t1,n0,m0,infiltratingductalcarcinoma,NaN,LumA,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,426.0,0.0,"426,0"
1,TCGA.OL.A5RW,40.0,0.8121,stageii,t1,n1,NaN,infiltratingductalcarcinoma,1.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,NaN,0.0,1106.0,0.0,"1106,0"
2,TCGA.XX.A899,46.0,0.6069,stageiii,t1,n2,NaN,infiltratinglobularcarcinoma,5.0,NaN,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,467.0,0.0,"467,0"
3,TCGA.S3.A6ZH,29.0,0.8698,stageiii,t3,n3,NaN,infiltratingductalcarcinoma,14.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,641.0,0.0,"641,0"
4,TCGA.AQ.A54N,51.0,0.9075,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,Basal,...,NaN,NaN,female,no,blackorafricanamerican,hispanicorlatino,0.0,78.0,0.0,"78,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,TCGA.GM.A5PX,65.0,0.7913,stageii,t2,n1,m0,infiltratinglobularcarcinoma,1.0,NaN,...,NaN,NaN,female,no,white,nothispanicorlatino,0.0,551.0,0.0,"551,0"
766,TCGA.AC.A5XU,74.0,0.7798,stageii,t3,n0,NaN,infiltratingductalcarcinoma,0.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,455.0,0.0,"455,0"
767,TCGA.AR.A1AN,46.0,0.6708,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,LumA,...,NaN,NaN,female,yes,white,hispanicorlatino,0.0,2920.0,0.0,"2920,0"
768,TCGA.E9.A1RG,62.0,0.7904,stageiii,t1,n2,m0,infiltratingductalcarcinoma,4.0,NaN,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,647.0,0.0,"647,0"


In [24]:
# 读取制表符分隔的CSV文件
Methylation = pd.read_csv('../KIPAN_Methylation.csv')
# 显示 DataFrame
Methylation

MemoryError: 

In [ ]:
# 读取制表符分隔的CSV文件
SCNA = pd.read_csv('../../DATA/BRCA/BRCA_SCNA.csv')
# 显示 DataFrame
SCNA

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC46_row_2056,PC47_row_2056,PC48_row_2056,PC49_row_2056,PC50_row_2056,PC51_row_2056,PC52_row_2056,PC53_row_2056,PC54_row_2056,PC55_row_2056
0,-3.689608,-0.012004,0.740888,0.544708,-1.489282,0.693156,-0.547687,-0.135327,-0.680514,-0.312020,...,-0.002941,-0.065707,-0.029548,-0.103796,0.094434,-0.007562,0.030936,-0.036458,0.012247,0.026713
1,3.926256,-2.110920,4.536738,3.067629,-0.222910,-1.914895,2.193465,-2.189253,-0.034102,1.647097,...,0.144131,0.377514,0.357360,-0.483268,-0.362459,0.672465,-0.483449,0.024481,-0.206802,-0.317517
2,-2.197348,-0.079732,0.052953,-0.237772,-0.140364,-0.569511,0.411215,0.269261,-1.042993,-1.418734,...,-0.043357,0.038646,-0.035653,-0.004350,-0.042631,0.092868,-0.074244,-0.030044,0.043900,0.087815
3,3.047622,-0.771998,-3.650549,-1.955169,2.333739,0.555907,2.852939,0.209406,-2.643802,0.347106,...,-0.295449,0.038720,-0.055692,0.157902,-0.076535,0.110535,-0.183472,-0.338641,0.178137,-0.009920
4,3.510597,-2.084858,4.741810,-1.005164,2.153028,-3.022992,-0.087179,-1.045735,-4.950535,2.954984,...,0.465638,0.024485,0.035718,-0.354543,0.211643,-0.292999,0.078933,0.105181,0.395971,0.022695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,1.906514,-2.374485,0.158865,-0.605034,-0.243116,-0.414706,-1.149678,0.330166,0.244346,-0.754586,...,0.125299,0.010732,-0.013350,-0.086237,0.034113,-0.019802,0.079759,0.113044,-0.213753,-0.091645
766,0.212511,1.078317,-0.326279,-1.543744,0.476989,2.080515,0.674680,1.077134,3.023376,0.141956,...,-0.619378,0.387367,0.140058,-0.126219,0.294643,-0.350489,-0.413271,-0.286208,-0.366831,-0.194023
767,-1.467740,-2.456650,1.238528,-0.989609,0.198856,-0.126432,-0.267650,-0.303727,1.583198,-0.984309,...,-0.773998,-0.099146,0.176743,-0.135162,-0.543647,-0.149395,-0.129412,-0.181850,-0.086134,-0.256794
768,-0.611175,1.189737,-2.294920,-2.694414,0.643169,0.357134,0.918768,1.341373,0.958065,1.794307,...,-0.099333,-0.364067,0.204714,0.057028,0.081182,-0.159150,0.166607,0.240556,-0.581589,-0.293561


In [ ]:
# 读取制表符分隔的CSV文件
RNAseq = pd.read_csv('../../DATA/BRCA/BRCA_RNAseq.csv')
# 显示 DataFrame
RNAseq

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC22_row_2888,PC23_row_2888,PC24_row_2888,PC25_row_2888,PC26_row_2888,PC27_row_2888,PC28_row_2888,PC29_row_2888,PC30_row_2888,PC31_row_2888
0,-3.285974,-12.562589,-3.646209,0.386229,0.657875,-1.678248,-1.257449,-0.904046,1.464052,1.700234,...,-0.186164,-0.204501,0.314977,-0.060763,0.424976,-0.337710,0.485089,0.592095,-0.181426,0.204965
1,23.119830,12.508496,8.832580,5.551372,1.865155,1.417451,-3.523612,0.397494,-0.875563,-1.106149,...,1.088211,0.855416,0.873833,-0.930448,-0.274407,0.828816,-1.503358,-0.229105,1.351993,0.490903
2,0.992969,-10.714355,2.500595,-1.967800,3.036194,-1.814885,5.261554,-3.789177,-0.211309,-3.995686,...,0.348824,-0.247749,-0.485108,0.327940,-0.310485,0.108346,0.057962,0.404241,0.188288,-0.363698
3,-10.600821,9.065305,-0.362642,6.203554,-7.590398,-1.437865,-2.699939,-3.010715,3.898215,1.133414,...,0.214204,-0.076964,1.164457,-0.677932,0.742100,1.080115,-0.012995,0.550527,-0.990089,-0.665073
4,19.308543,21.275776,5.958776,10.156485,3.519821,4.387551,-7.663056,1.028869,-2.842409,2.037368,...,-2.067729,-0.778712,-1.581554,0.461892,-0.235884,-0.809221,-1.674960,1.981832,0.103678,-0.600245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-1.984872,-4.582652,10.426545,-2.225526,5.561229,0.593622,-1.165100,2.282577,-4.020406,3.891008,...,0.028549,0.960110,0.244905,0.573216,-1.205461,-0.097943,-0.219461,-0.381182,-0.990756,0.159133
766,-7.781401,3.008800,2.408835,1.052876,0.781646,-11.131030,5.443650,6.573161,2.205694,-5.237724,...,-1.098735,0.934456,0.058153,0.281565,-0.612909,-0.345822,0.156467,0.791132,0.798751,0.484092
767,-6.277986,-6.473330,-2.050684,-3.790173,2.480567,-1.326708,-2.411364,-3.708606,2.013082,3.979026,...,0.359238,-0.083191,-0.290969,0.031183,0.080596,0.573752,-0.817779,-0.040454,0.029639,0.244322
768,-10.363677,11.075287,11.949353,-7.649157,-2.399230,-1.966285,-4.681857,-4.669735,-1.512971,-0.227234,...,-0.804612,-0.387581,0.183021,-0.736157,-0.646209,0.501357,0.165683,-0.734769,0.633304,0.642659


In [ ]:
# 找到Clinical中包含NaN值的行的索引
nan_indices = Clinical[Clinical.iloc[:, -2].isna()].index

In [ ]:
# 删除Clinical中包含NaN值的行
Clinical.dropna(subset=[Clinical.columns[-2]], inplace=True)
Clinical

,Unnamed: 0,years_to_birth,Tumor_purity,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,histological_type,number_of_lymph_nodes,PAM50,...,PR.Status,HER2.Status,gender,radiation_therapy,race,ethnicity,Median_overall_survival,overall_survival,status,overallsurvival
0,TCGA.D8.A1JH,56.0,0.6126,stagei,t1,n0,m0,infiltratingductalcarcinoma,NaN,LumA,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,426.0,0.0,"426,0"
1,TCGA.OL.A5RW,40.0,0.8121,stageii,t1,n1,NaN,infiltratingductalcarcinoma,1.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,NaN,0.0,1106.0,0.0,"1106,0"
2,TCGA.XX.A899,46.0,0.6069,stageiii,t1,n2,NaN,infiltratinglobularcarcinoma,5.0,NaN,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,467.0,0.0,"467,0"
3,TCGA.S3.A6ZH,29.0,0.8698,stageiii,t3,n3,NaN,infiltratingductalcarcinoma,14.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,641.0,0.0,"641,0"
4,TCGA.AQ.A54N,51.0,0.9075,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,Basal,...,NaN,NaN,female,no,blackorafricanamerican,hispanicorlatino,0.0,78.0,0.0,"78,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,TCGA.GM.A5PX,65.0,0.7913,stageii,t2,n1,m0,infiltratinglobularcarcinoma,1.0,NaN,...,NaN,NaN,female,no,white,nothispanicorlatino,0.0,551.0,0.0,"551,0"
766,TCGA.AC.A5XU,74.0,0.7798,stageii,t3,n0,NaN,infiltratingductalcarcinoma,0.0,NaN,...,NaN,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,455.0,0.0,"455,0"
767,TCGA.AR.A1AN,46.0,0.6708,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,LumA,...,NaN,NaN,female,yes,white,hispanicorlatino,0.0,2920.0,0.0,"2920,0"
768,TCGA.E9.A1RG,62.0,0.7904,stageiii,t1,n2,m0,infiltratingductalcarcinoma,4.0,NaN,...,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,647.0,0.0,"647,0"


In [ ]:
# 如果需要在原DataFrame中删除这些行
Methylation.drop(index=nan_indices, inplace=True)
Methylation

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC23_row_2785,PC24_row_2785,PC25_row_2785,PC26_row_2785,PC27_row_2785,PC28_row_2785,PC29_row_2785,PC30_row_2785,PC31_row_2785,PC32_row_2785
0,-0.153909,-0.321220,-0.643329,0.031471,0.025801,-0.017464,-0.123185,0.364893,-0.064358,-0.256289,...,0.027432,0.110140,-0.081281,0.359896,0.287192,-0.007336,-0.090472,0.002717,0.200100,-0.105640
1,-2.607611,-0.579033,1.383772,-0.395870,0.305295,0.423947,-0.022199,0.523331,-0.202012,-0.048350,...,-0.035319,-0.073902,0.133344,-0.303436,0.050312,0.087587,-0.148580,-0.138114,0.023566,0.073351
2,-0.371162,0.329574,-0.346730,-0.113366,-0.385165,0.224095,-0.314062,-0.202690,-0.408260,0.152930,...,-0.075493,-0.095261,-0.059597,0.039381,-0.028532,0.001307,-0.030177,0.014623,0.032871,-0.058927
3,1.045107,-1.641925,0.566927,0.902616,0.266469,-0.135588,-0.241015,-0.491515,-0.086686,0.207742,...,-0.127915,-0.202101,0.211982,0.052114,0.037772,0.034079,-0.141561,0.117653,0.053892,-0.060446
4,-2.002543,-0.293929,3.001267,-0.138371,-0.314426,0.156589,0.641048,-0.512697,-0.001995,-0.211870,...,0.067536,-0.101198,-0.245086,-0.154037,-0.114525,0.092898,-0.073257,-0.281584,0.029717,0.160549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-0.049419,-0.176050,-0.372327,-0.597913,-0.601829,0.316945,-0.360133,-0.300012,0.234291,0.035366,...,-0.006785,0.079181,-0.149082,-0.063396,0.057082,0.021570,0.031578,-0.062265,-0.001039,-0.000067
766,1.829325,0.292855,0.336571,0.534994,0.249550,-0.394503,-0.224960,-0.225572,-0.229210,0.344383,...,0.053151,-0.088975,0.119671,-0.032908,-0.095264,0.089833,-0.044777,0.155025,-0.135487,0.046864
767,-0.603756,-1.215633,-0.128903,-0.001386,-0.564716,-0.199250,0.203827,0.217636,-0.149666,0.072106,...,0.116306,0.185560,-0.054880,0.231036,-0.150081,0.019259,-0.065327,0.209717,0.083584,-0.034588
768,-0.456618,-1.115913,-0.511820,0.326681,0.758917,-0.372317,-0.370912,-0.285424,0.402555,-0.056596,...,-0.006683,-0.085667,-0.110781,0.035284,-0.101247,-0.025504,0.013511,-0.032645,0.029408,-0.108519


In [ ]:
# 过滤掉没有信号或低方差的特征
Methylation = filter_features(Methylation)
Methylation

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC23_row_2785,PC24_row_2785,PC25_row_2785,PC26_row_2785,PC27_row_2785,PC28_row_2785,PC29_row_2785,PC30_row_2785,PC31_row_2785,PC32_row_2785
0,-0.153909,-0.321220,-0.643329,0.031471,0.025801,-0.017464,-0.123185,0.364893,-0.064358,-0.256289,...,0.027432,0.110140,-0.081281,0.359896,0.287192,-0.007336,-0.090472,0.002717,0.200100,-0.105640
1,-2.607611,-0.579033,1.383772,-0.395870,0.305295,0.423947,-0.022199,0.523331,-0.202012,-0.048350,...,-0.035319,-0.073902,0.133344,-0.303436,0.050312,0.087587,-0.148580,-0.138114,0.023566,0.073351
2,-0.371162,0.329574,-0.346730,-0.113366,-0.385165,0.224095,-0.314062,-0.202690,-0.408260,0.152930,...,-0.075493,-0.095261,-0.059597,0.039381,-0.028532,0.001307,-0.030177,0.014623,0.032871,-0.058927
3,1.045107,-1.641925,0.566927,0.902616,0.266469,-0.135588,-0.241015,-0.491515,-0.086686,0.207742,...,-0.127915,-0.202101,0.211982,0.052114,0.037772,0.034079,-0.141561,0.117653,0.053892,-0.060446
4,-2.002543,-0.293929,3.001267,-0.138371,-0.314426,0.156589,0.641048,-0.512697,-0.001995,-0.211870,...,0.067536,-0.101198,-0.245086,-0.154037,-0.114525,0.092898,-0.073257,-0.281584,0.029717,0.160549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-0.049419,-0.176050,-0.372327,-0.597913,-0.601829,0.316945,-0.360133,-0.300012,0.234291,0.035366,...,-0.006785,0.079181,-0.149082,-0.063396,0.057082,0.021570,0.031578,-0.062265,-0.001039,-0.000067
766,1.829325,0.292855,0.336571,0.534994,0.249550,-0.394503,-0.224960,-0.225572,-0.229210,0.344383,...,0.053151,-0.088975,0.119671,-0.032908,-0.095264,0.089833,-0.044777,0.155025,-0.135487,0.046864
767,-0.603756,-1.215633,-0.128903,-0.001386,-0.564716,-0.199250,0.203827,0.217636,-0.149666,0.072106,...,0.116306,0.185560,-0.054880,0.231036,-0.150081,0.019259,-0.065327,0.209717,0.083584,-0.034588
768,-0.456618,-1.115913,-0.511820,0.326681,0.758917,-0.372317,-0.370912,-0.285424,0.402555,-0.056596,...,-0.006683,-0.085667,-0.110781,0.035284,-0.101247,-0.025504,0.013511,-0.032645,0.029408,-0.108519


In [ ]:
Methylation = select_features(Methylation, Clinical.iloc[:,-2])

# Output selected features DataFrame
print("Selected features after FDR correction:")
Methylation

Selected features after FDR correction:


,PC26_row_18,PC3_row_34,PC24_row_65,PC15_row_74,PC24_row_83,PC3_row_168,PC14_row_171,PC84_row_173,PC4_row_186,PC3_row_192,...,PC8_row_2177,PC3_row_2246,PC10_row_2304,PC3_row_2309,PC12_row_2375,PC3_row_2385,PC3_row_2499,PC3_row_2575,PC3_row_2583,PC4_row_2702
0,-0.114922,-0.411908,-0.108683,-0.018514,-0.180296,0.029649,0.050315,0.251495,-0.683849,-0.211469,...,0.202261,0.018806,-0.021598,-0.184433,-0.398926,-0.173010,-0.159878,-0.020807,-0.003930,-0.045792
1,-0.286586,0.142205,-0.227236,-0.113588,0.024147,0.456946,0.036837,0.244036,0.804284,-0.054208,...,-0.238017,-0.042860,-0.141648,-0.063028,0.055239,0.098332,-0.027299,0.265673,-0.014667,0.125894
2,0.021096,-0.620292,0.243027,0.088556,-0.005869,-0.148066,-0.001065,0.019197,-0.391563,-0.478830,...,0.063726,0.050004,0.153113,-0.129982,-0.088167,-0.058496,-0.137275,-0.098938,0.048742,0.169430
3,-0.387332,0.509461,-0.092411,-0.365419,-0.086260,0.573632,-0.103787,0.189387,0.989876,0.507632,...,0.016207,-0.014856,-0.160045,-0.388962,0.033549,-0.179527,-0.408337,0.372213,-0.035996,-0.347927
4,-0.149353,0.863226,-0.190764,-0.431092,0.132531,0.407208,-0.103519,0.024833,1.813941,0.727028,...,0.124658,0.622632,-0.138923,0.084463,0.067980,0.374625,0.113022,0.490986,0.630249,-0.396256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,0.127734,-0.484438,0.042187,0.030010,-0.049157,-0.397902,0.116442,0.136478,-0.603806,-0.490658,...,-0.043846,0.021768,0.077379,-0.141668,0.085375,-0.216796,-0.119987,0.036023,0.016253,-0.214953
766,-0.487799,0.631100,-0.117413,0.197093,-0.309131,0.177384,-0.160612,-0.025787,0.419215,0.356970,...,-0.183681,0.073333,0.260046,-0.066160,0.111991,-0.021324,-0.137184,0.074069,0.059454,0.367999
767,0.201043,-0.660310,0.292462,0.145352,0.230128,-0.082456,-0.051484,0.072141,0.113571,-0.583538,...,0.174896,0.030308,-0.289011,0.164062,0.088255,0.096979,0.156168,-0.293502,0.026219,0.161473
768,0.117406,0.176982,-0.165084,0.120163,0.134083,-0.119821,-0.652007,0.002598,0.425936,0.217074,...,-0.131035,-0.073469,-0.196078,-0.035714,0.140977,-0.222143,-0.075016,0.211943,-0.087382,-0.121387


In [ ]:
Methylation = scale_to_unit_interval(Methylation)
Methylation

,PC26_row_18,PC3_row_34,PC24_row_65,PC15_row_74,PC24_row_83,PC3_row_168,PC14_row_171,PC84_row_173,PC4_row_186,PC3_row_192,...,PC8_row_2177,PC3_row_2246,PC10_row_2304,PC3_row_2309,PC12_row_2375,PC3_row_2385,PC3_row_2499,PC3_row_2575,PC3_row_2583,PC4_row_2702
0,0.373168,0.222183,0.405108,0.455262,0.358327,0.398201,0.464927,0.758550,0.175752,0.268490,...,0.625668,0.441709,0.462210,0.289291,0.000000,0.281535,0.320237,0.348022,0.441024,0.377765
1,0.292761,0.415277,0.320345,0.395271,0.488760,0.589769,0.456825,0.748212,0.544954,0.330148,...,0.234486,0.406123,0.359851,0.355935,0.567613,0.427599,0.402907,0.559303,0.434908,0.560721
2,0.436878,0.149567,0.656574,0.522823,0.469610,0.318527,0.434042,0.436581,0.248267,0.163664,...,0.502581,0.459712,0.611175,0.319182,0.388385,0.343179,0.334332,0.290400,0.471028,0.607114
3,0.245571,0.543257,0.416742,0.236365,0.418322,0.642082,0.372294,0.672467,0.590999,0.550432,...,0.460361,0.422284,0.344165,0.177018,0.540505,0.278027,0.165310,0.637877,0.422758,0.055800
4,0.357040,0.666535,0.346421,0.194926,0.557908,0.567470,0.372455,0.444393,0.795447,0.636452,...,0.556718,0.790159,0.362174,0.436898,0.583537,0.576328,0.490405,0.725473,0.802274,0.004298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,0.486827,0.196908,0.512976,0.485881,0.441992,0.206519,0.504678,0.599134,0.195611,0.159027,...,0.407005,0.443418,0.546601,0.312767,0.605276,0.257966,0.345112,0.389934,0.452521,0.197501
766,0.198513,0.585645,0.398866,0.591310,0.276132,0.464434,0.338135,0.374233,0.449419,0.491361,...,0.282764,0.473175,0.702349,0.354216,0.638541,0.363188,0.334388,0.417994,0.477130,0.818718
767,0.521165,0.135621,0.691919,0.558662,0.620174,0.347941,0.403734,0.509963,0.373590,0.122610,...,0.601354,0.448346,0.234204,0.480593,0.608876,0.426871,0.517309,0.146908,0.458198,0.598635
768,0.481989,0.427396,0.364782,0.542768,0.558898,0.331190,0.042748,0.413575,0.451087,0.436512,...,0.329539,0.388460,0.313442,0.370928,0.674768,0.255087,0.373154,0.519677,0.393487,0.297209


In [ ]:
# 如果需要在原DataFrame中删除这些行
SCNA.drop(index=nan_indices, inplace=True)
SCNA

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC46_row_2056,PC47_row_2056,PC48_row_2056,PC49_row_2056,PC50_row_2056,PC51_row_2056,PC52_row_2056,PC53_row_2056,PC54_row_2056,PC55_row_2056
0,-3.689608,-0.012004,0.740888,0.544708,-1.489282,0.693156,-0.547687,-0.135327,-0.680514,-0.312020,...,-0.002941,-0.065707,-0.029548,-0.103796,0.094434,-0.007562,0.030936,-0.036458,0.012247,0.026713
1,3.926256,-2.110920,4.536738,3.067629,-0.222910,-1.914895,2.193465,-2.189253,-0.034102,1.647097,...,0.144131,0.377514,0.357360,-0.483268,-0.362459,0.672465,-0.483449,0.024481,-0.206802,-0.317517
2,-2.197348,-0.079732,0.052953,-0.237772,-0.140364,-0.569511,0.411215,0.269261,-1.042993,-1.418734,...,-0.043357,0.038646,-0.035653,-0.004350,-0.042631,0.092868,-0.074244,-0.030044,0.043900,0.087815
3,3.047622,-0.771998,-3.650549,-1.955169,2.333739,0.555907,2.852939,0.209406,-2.643802,0.347106,...,-0.295449,0.038720,-0.055692,0.157902,-0.076535,0.110535,-0.183472,-0.338641,0.178137,-0.009920
4,3.510597,-2.084858,4.741810,-1.005164,2.153028,-3.022992,-0.087179,-1.045735,-4.950535,2.954984,...,0.465638,0.024485,0.035718,-0.354543,0.211643,-0.292999,0.078933,0.105181,0.395971,0.022695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,1.906514,-2.374485,0.158865,-0.605034,-0.243116,-0.414706,-1.149678,0.330166,0.244346,-0.754586,...,0.125299,0.010732,-0.013350,-0.086237,0.034113,-0.019802,0.079759,0.113044,-0.213753,-0.091645
766,0.212511,1.078317,-0.326279,-1.543744,0.476989,2.080515,0.674680,1.077134,3.023376,0.141956,...,-0.619378,0.387367,0.140058,-0.126219,0.294643,-0.350489,-0.413271,-0.286208,-0.366831,-0.194023
767,-1.467740,-2.456650,1.238528,-0.989609,0.198856,-0.126432,-0.267650,-0.303727,1.583198,-0.984309,...,-0.773998,-0.099146,0.176743,-0.135162,-0.543647,-0.149395,-0.129412,-0.181850,-0.086134,-0.256794
768,-0.611175,1.189737,-2.294920,-2.694414,0.643169,0.357134,0.918768,1.341373,0.958065,1.794307,...,-0.099333,-0.364067,0.204714,0.057028,0.081182,-0.159150,0.166607,0.240556,-0.581589,-0.293561


In [ ]:
# 过滤掉没有信号或低方差的特征
SCNA = filter_features(SCNA)
SCNA

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC46_row_2056,PC47_row_2056,PC48_row_2056,PC49_row_2056,PC50_row_2056,PC51_row_2056,PC52_row_2056,PC53_row_2056,PC54_row_2056,PC55_row_2056
0,-3.689608,-0.012004,0.740888,0.544708,-1.489282,0.693156,-0.547687,-0.135327,-0.680514,-0.312020,...,-0.002941,-0.065707,-0.029548,-0.103796,0.094434,-0.007562,0.030936,-0.036458,0.012247,0.026713
1,3.926256,-2.110920,4.536738,3.067629,-0.222910,-1.914895,2.193465,-2.189253,-0.034102,1.647097,...,0.144131,0.377514,0.357360,-0.483268,-0.362459,0.672465,-0.483449,0.024481,-0.206802,-0.317517
2,-2.197348,-0.079732,0.052953,-0.237772,-0.140364,-0.569511,0.411215,0.269261,-1.042993,-1.418734,...,-0.043357,0.038646,-0.035653,-0.004350,-0.042631,0.092868,-0.074244,-0.030044,0.043900,0.087815
3,3.047622,-0.771998,-3.650549,-1.955169,2.333739,0.555907,2.852939,0.209406,-2.643802,0.347106,...,-0.295449,0.038720,-0.055692,0.157902,-0.076535,0.110535,-0.183472,-0.338641,0.178137,-0.009920
4,3.510597,-2.084858,4.741810,-1.005164,2.153028,-3.022992,-0.087179,-1.045735,-4.950535,2.954984,...,0.465638,0.024485,0.035718,-0.354543,0.211643,-0.292999,0.078933,0.105181,0.395971,0.022695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,1.906514,-2.374485,0.158865,-0.605034,-0.243116,-0.414706,-1.149678,0.330166,0.244346,-0.754586,...,0.125299,0.010732,-0.013350,-0.086237,0.034113,-0.019802,0.079759,0.113044,-0.213753,-0.091645
766,0.212511,1.078317,-0.326279,-1.543744,0.476989,2.080515,0.674680,1.077134,3.023376,0.141956,...,-0.619378,0.387367,0.140058,-0.126219,0.294643,-0.350489,-0.413271,-0.286208,-0.366831,-0.194023
767,-1.467740,-2.456650,1.238528,-0.989609,0.198856,-0.126432,-0.267650,-0.303727,1.583198,-0.984309,...,-0.773998,-0.099146,0.176743,-0.135162,-0.543647,-0.149395,-0.129412,-0.181850,-0.086134,-0.256794
768,-0.611175,1.189737,-2.294920,-2.694414,0.643169,0.357134,0.918768,1.341373,0.958065,1.794307,...,-0.099333,-0.364067,0.204714,0.057028,0.081182,-0.159150,0.166607,0.240556,-0.581589,-0.293561


In [ ]:
SCNA = select_features(SCNA, Clinical.iloc[:,-2])

# Output selected features DataFrame
print("Selected features after FDR correction:")
SCNA

Selected features after FDR correction:


""
0
1
2
3
4
...
765
766
767
768


In [ ]:
SCNA = scale_to_unit_interval(SCNA)
SCNA

""
0
1
2
3
4
...
765
766
767
768


In [ ]:
# 如果需要在原DataFrame中删除这些行
RNAseq.drop(index=nan_indices, inplace=True)
RNAseq

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC22_row_2888,PC23_row_2888,PC24_row_2888,PC25_row_2888,PC26_row_2888,PC27_row_2888,PC28_row_2888,PC29_row_2888,PC30_row_2888,PC31_row_2888
0,-3.285974,-12.562589,-3.646209,0.386229,0.657875,-1.678248,-1.257449,-0.904046,1.464052,1.700234,...,-0.186164,-0.204501,0.314977,-0.060763,0.424976,-0.337710,0.485089,0.592095,-0.181426,0.204965
1,23.119830,12.508496,8.832580,5.551372,1.865155,1.417451,-3.523612,0.397494,-0.875563,-1.106149,...,1.088211,0.855416,0.873833,-0.930448,-0.274407,0.828816,-1.503358,-0.229105,1.351993,0.490903
2,0.992969,-10.714355,2.500595,-1.967800,3.036194,-1.814885,5.261554,-3.789177,-0.211309,-3.995686,...,0.348824,-0.247749,-0.485108,0.327940,-0.310485,0.108346,0.057962,0.404241,0.188288,-0.363698
3,-10.600821,9.065305,-0.362642,6.203554,-7.590398,-1.437865,-2.699939,-3.010715,3.898215,1.133414,...,0.214204,-0.076964,1.164457,-0.677932,0.742100,1.080115,-0.012995,0.550527,-0.990089,-0.665073
4,19.308543,21.275776,5.958776,10.156485,3.519821,4.387551,-7.663056,1.028869,-2.842409,2.037368,...,-2.067729,-0.778712,-1.581554,0.461892,-0.235884,-0.809221,-1.674960,1.981832,0.103678,-0.600245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-1.984872,-4.582652,10.426545,-2.225526,5.561229,0.593622,-1.165100,2.282577,-4.020406,3.891008,...,0.028549,0.960110,0.244905,0.573216,-1.205461,-0.097943,-0.219461,-0.381182,-0.990756,0.159133
766,-7.781401,3.008800,2.408835,1.052876,0.781646,-11.131030,5.443650,6.573161,2.205694,-5.237724,...,-1.098735,0.934456,0.058153,0.281565,-0.612909,-0.345822,0.156467,0.791132,0.798751,0.484092
767,-6.277986,-6.473330,-2.050684,-3.790173,2.480567,-1.326708,-2.411364,-3.708606,2.013082,3.979026,...,0.359238,-0.083191,-0.290969,0.031183,0.080596,0.573752,-0.817779,-0.040454,0.029639,0.244322
768,-10.363677,11.075287,11.949353,-7.649157,-2.399230,-1.966285,-4.681857,-4.669735,-1.512971,-0.227234,...,-0.804612,-0.387581,0.183021,-0.736157,-0.646209,0.501357,0.165683,-0.734769,0.633304,0.642659


In [ ]:
# 过滤掉没有信号或低方差的特征
RNAseq = filter_features(RNAseq)
RNAseq

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC22_row_2888,PC23_row_2888,PC24_row_2888,PC25_row_2888,PC26_row_2888,PC27_row_2888,PC28_row_2888,PC29_row_2888,PC30_row_2888,PC31_row_2888
0,-3.285974,-12.562589,-3.646209,0.386229,0.657875,-1.678248,-1.257449,-0.904046,1.464052,1.700234,...,-0.186164,-0.204501,0.314977,-0.060763,0.424976,-0.337710,0.485089,0.592095,-0.181426,0.204965
1,23.119830,12.508496,8.832580,5.551372,1.865155,1.417451,-3.523612,0.397494,-0.875563,-1.106149,...,1.088211,0.855416,0.873833,-0.930448,-0.274407,0.828816,-1.503358,-0.229105,1.351993,0.490903
2,0.992969,-10.714355,2.500595,-1.967800,3.036194,-1.814885,5.261554,-3.789177,-0.211309,-3.995686,...,0.348824,-0.247749,-0.485108,0.327940,-0.310485,0.108346,0.057962,0.404241,0.188288,-0.363698
3,-10.600821,9.065305,-0.362642,6.203554,-7.590398,-1.437865,-2.699939,-3.010715,3.898215,1.133414,...,0.214204,-0.076964,1.164457,-0.677932,0.742100,1.080115,-0.012995,0.550527,-0.990089,-0.665073
4,19.308543,21.275776,5.958776,10.156485,3.519821,4.387551,-7.663056,1.028869,-2.842409,2.037368,...,-2.067729,-0.778712,-1.581554,0.461892,-0.235884,-0.809221,-1.674960,1.981832,0.103678,-0.600245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-1.984872,-4.582652,10.426545,-2.225526,5.561229,0.593622,-1.165100,2.282577,-4.020406,3.891008,...,0.028549,0.960110,0.244905,0.573216,-1.205461,-0.097943,-0.219461,-0.381182,-0.990756,0.159133
766,-7.781401,3.008800,2.408835,1.052876,0.781646,-11.131030,5.443650,6.573161,2.205694,-5.237724,...,-1.098735,0.934456,0.058153,0.281565,-0.612909,-0.345822,0.156467,0.791132,0.798751,0.484092
767,-6.277986,-6.473330,-2.050684,-3.790173,2.480567,-1.326708,-2.411364,-3.708606,2.013082,3.979026,...,0.359238,-0.083191,-0.290969,0.031183,0.080596,0.573752,-0.817779,-0.040454,0.029639,0.244322
768,-10.363677,11.075287,11.949353,-7.649157,-2.399230,-1.966285,-4.681857,-4.669735,-1.512971,-0.227234,...,-0.804612,-0.387581,0.183021,-0.736157,-0.646209,0.501357,0.165683,-0.734769,0.633304,0.642659


In [ ]:
RNAseq = select_features(RNAseq, Clinical.iloc[:,-2])

# Output selected features DataFrame
print("Selected features after FDR correction:")
RNAseq

Selected features after FDR correction:


,PC69_row_24,PC17_row_95,PC42_row_443,PC22_row_499,PC4_row_604,PC6_row_654,PC10_row_665,PC4_row_752,PC18_row_824,PC11_row_906,...,PC10_row_1532,PC10_row_1533,PC11_row_1743,PC13_row_1769,PC14_row_2103,PC12_row_2573,PC57_row_2597,PC8_row_2851,PC8_row_2854,PC8_row_2855
0,-0.610412,0.031798,-0.785846,1.163333,0.987689,-0.510918,0.687965,0.831083,-0.086835,0.171947,...,-0.156747,-0.156730,-0.049212,0.204870,-0.139946,-0.030888,0.039598,0.012267,0.012267,0.012267
1,-0.144128,0.959367,-2.930142,2.820159,0.727732,1.002601,-0.302499,1.200991,0.670207,-0.948455,...,4.349822,4.349726,-0.861753,-1.743404,-0.760153,1.576209,-1.172771,-0.310572,-0.310572,-0.310572
2,0.904578,-1.150918,-1.686078,-1.346674,-2.482038,-2.682121,3.438077,-2.922308,-0.451005,0.975801,...,-0.949236,-0.949266,0.415875,-0.014328,0.423336,-0.214340,0.983123,0.224174,0.224174,0.224174
3,2.319150,-1.745110,-1.560396,2.833424,1.475332,-2.350615,-0.781307,1.358660,-0.282861,2.401299,...,1.559331,1.559327,0.232133,-0.897075,-1.648078,0.794568,-0.976755,-0.753989,-0.753989,-0.753989
4,1.391851,0.805876,3.123773,-2.928945,-3.028476,3.655518,1.753408,-2.652807,0.241215,-3.811685,...,0.620559,0.620454,-1.049891,-1.674768,-1.243155,0.370780,-0.195591,0.687836,0.687836,0.687836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,-0.060143,0.193044,0.478065,0.115011,-4.232516,2.633359,-1.710116,-4.383484,-1.200211,-0.197351,...,-0.965450,-0.965507,-1.879799,1.041708,-0.353552,0.341897,-0.270329,0.600026,0.600026,0.600026
766,0.374353,1.237212,-0.409767,0.385845,-3.455338,-0.966813,-0.671235,-3.579045,-1.254537,-2.142353,...,3.189744,3.189829,0.167881,-1.001668,-0.431559,0.992441,0.398973,-0.438537,-0.438537,-0.438537
767,2.172273,1.842877,-0.114228,-0.290332,-0.733478,0.744630,1.343292,-0.787265,-0.019456,1.035672,...,-0.176428,-0.176466,0.738410,-0.491252,1.380924,-0.107356,0.585823,-0.722598,-0.722598,-0.722598
768,-1.112003,1.083012,0.983990,1.343706,0.350552,-1.149124,-0.875997,0.320478,0.208161,-0.526220,...,-0.045496,-0.045533,1.834394,-0.820362,0.679317,-0.140551,0.170774,0.345780,0.345780,0.345780


In [ ]:
RNAseq = scale_to_unit_interval(RNAseq)
RNAseq

,PC69_row_24,PC17_row_95,PC42_row_443,PC22_row_499,PC4_row_604,PC6_row_654,PC10_row_665,PC4_row_752,PC18_row_824,PC11_row_906,...,PC10_row_1532,PC10_row_1533,PC11_row_1743,PC13_row_1769,PC14_row_2103,PC12_row_2573,PC57_row_2597,PC8_row_2851,PC8_row_2854,PC8_row_2855
0,0.403808,0.482093,0.413216,0.611549,0.492742,0.462872,0.492221,0.510466,0.453995,0.508924,...,0.448961,0.448970,0.430658,0.517066,0.446605,0.468786,0.480016,0.433837,0.433837,0.433837
1,0.460391,0.550750,0.205641,0.826845,0.476090,0.544332,0.436279,0.533687,0.593773,0.396914,...,1.000000,1.000000,0.363551,0.197357,0.324570,0.862871,0.248731,0.351213,0.351213,0.351213
2,0.587650,0.394551,0.326071,0.285386,0.270486,0.346015,0.647549,0.274845,0.386756,0.589287,...,0.352060,0.352062,0.469069,0.481096,0.557440,0.423800,0.660013,0.488070,0.488070,0.488070
3,0.759306,0.350570,0.338237,0.828569,0.523978,0.363857,0.409235,0.543585,0.417802,0.731799,...,0.658794,0.658802,0.453894,0.336238,0.149856,0.671200,0.286125,0.237730,0.237730,0.237730
4,0.646780,0.539388,0.791679,0.079777,0.235483,0.687117,0.552398,0.291763,0.514565,0.110668,...,0.544006,0.544000,0.348013,0.208620,0.229531,0.567281,0.435149,0.606735,0.606735,0.606735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,0.470583,0.494028,0.535567,0.475324,0.158357,0.632102,0.356776,0.183119,0.248425,0.472004,...,0.350077,0.350076,0.279472,0.654391,0.404575,0.560198,0.420891,0.584262,0.584262,0.584262
766,0.523308,0.571315,0.449622,0.510518,0.208140,0.438335,0.415452,0.233618,0.238394,0.277556,...,0.858152,0.858173,0.448587,0.319075,0.389226,0.719722,0.548574,0.318463,0.318463,0.318463
767,0.741483,0.616145,0.478231,0.422652,0.382491,0.530448,0.529234,0.408873,0.466436,0.595273,...,0.446555,0.446556,0.495707,0.402833,0.745860,0.450034,0.584219,0.245764,0.245764,0.245764
768,0.342941,0.559901,0.584542,0.634987,0.451929,0.428523,0.403887,0.478412,0.508462,0.439126,...,0.462564,0.462566,0.586223,0.348827,0.607808,0.441895,0.505040,0.519193,0.519193,0.519193


In [ ]:
X = Methylation
y = Clinical.iloc[:,-2]
Methylation_X_train, Methylation_X_test, Methylation_y_train, Methylation_y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Methylation_X_train, Methylation_X_val, Methylation_y_train, Methylation_y_val = train_test_split(Methylation_X_train, Methylation_y_train, test_size=0.2, random_state=42)

In [ ]:
Methylation_X_train.to_csv('../../DATA/BRCA/Methylation_train.csv', index=False)

In [ ]:
Methylation_X_test.to_csv('../../DATA/BRCA/Methylation_test.csv', index=False)

In [ ]:
# Methylation_X_val.to_csv('../../DATA/BRCA/Methylation_val.csv', index=False)

In [ ]:
Methylation_y_train.to_csv('../../DATA/BRCA/labels_train.csv', index=False)

In [ ]:
Methylation_y_test.to_csv('../../DATA/BRCA/labels_test.csv', index=False)

In [ ]:
# Methylation_y_val.to_csv('../../DATA/BRCA/labels_val.csv', index=False)

In [ ]:
X = SCNA
y = Clinical.iloc[:,-2]
SCNA_X_train, SCNA_X_test, SCNA_y_train, SCNA_y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# SCNA_X_train, SCNA_X_val, SCNA_y_train, SCNA_y_val = train_test_split(SCNA_X_train, SCNA_y_train, test_size=0.2, random_state=42)

In [ ]:
SCNA_X_train.to_csv('../../DATA/BRCA/SCNA_train.csv', index=False)

In [ ]:
SCNA_X_test.to_csv('../../DATA/BRCA/SCNA_test.csv', index=False)

In [ ]:
# SCNA_X_val.to_csv('../../DATA/BRCA/SCNA_val.csv', index=False)

In [ ]:
X = RNAseq
y = Clinical.iloc[:,-2]
RNAseq_X_train, RNAseq_X_test, RNAseq_y_train, RNAseq_y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# RNAseq_X_train, RNAseq_X_val, RNAseq_y_train, RNAseq_y_val = train_test_split(RNAseq_X_train, RNAseq_y_train, test_size=0.2, random_state=42)

In [ ]:
RNAseq_X_train.to_csv('../../DATA/BRCA/RNAseq_train.csv', index=False)

In [ ]:
RNAseq_X_test.to_csv('../../DATA/BRCA/RNAseq_test.csv', index=False)

In [ ]:
# RNAseq_X_val.to_csv('../../DATA/BRCA/RNAseq_val.csv', index=False)